In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0
!pip install folium

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
     newyork_data = json.load(json_data)
newyork_data

Solving environment: done

# All requested packages already installed.



{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [2]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [3]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [4]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [6]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [7]:
import folium
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
map_newyork

In [8]:
CLIENT_ID = 'I44Q3NWI4WWFMH5BMQP23Z0FGGFF0TD4J1HGPSZ4TGPBIBP4' 
CLIENT_SECRET = 'LCUELDEXV53QAL110ZNJ0RULFP3KMRLMUWROVQFVBBALFMAV' 
VERSION = '20180605' 
limit = 100

In [9]:
import requests
from pandas.io.json import json_normalize

def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)

        results = requests.get(url).json()['response']['groups'][0]['items']

        venues_list.append([(name,
                            lat,
                            lng,
                            v['venue']['name'],
                            v['venue']['location']['lat'],
                            v['venue']['location']['lng'],
                            v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                                  'Latitude', 
                                  'Longitude', 
                                  'Venue', 
                                  'Venue Latitude', 
                                  'Venue Longitude', 
                                  'Venue Category']
    return(nearby_venues)

In [10]:
LIMIT=100
def Extract_only_Restaurant(data_frame):
    df=data_frame
    df1=df[df['Venue Category'].str.contains("Restaurant")]
    df1.reset_index(inplace=True)
    df1=df1[['Neighborhood','Latitude','Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']]
    return(df1)

In [11]:
venues=getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],
                                   radius=500)

In [12]:
restaurants=Extract_only_Restaurant(venues)
restaurants

,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
1,Co-op City,40.874294,-73.829939,Arby's,40.870518,-73.828657,Fast Food Restaurant
2,Co-op City,40.874294,-73.829939,Townhouse Restaurant,40.876086,-73.828868,Restaurant
3,Eastchester,40.887556,-73.827806,Fish & Ting,40.885539,-73.829151,Caribbean Restaurant
4,Eastchester,40.887556,-73.827806,Dyre Fish Market,40.889318,-73.831453,Seafood Restaurant
5,Eastchester,40.887556,-73.827806,Golden Krust Caribbean Restaurant,40.888543,-73.831278,Caribbean Restaurant
6,Eastchester,40.887556,-73.827806,McDonald's,40.885636,-73.826840,Fast Food Restaurant
7,Eastchester,40.887556,-73.827806,Royal Caribbean Bakery,40.888252,-73.831457,Caribbean Restaurant
8,Eastchester,40.887556,-73.827806,Xing Lung Chinese Restaurant,40.888785,-73.831226,Chinese Restaurant
9,Kingsbridge,40.881687,-73.902818,Leche y Miel,40.883709,-73.901606,Latin American Restaurant


In [13]:
def Extract_only_Asian_Restaurant(data_frame):
    df=data_frame
    search_values=["Asian Restaurant", "Thai Restaurant", "Korean Restaurant", "Chinese Restaurant", "Japanese Restaurant", "Shanghai Restaurant", "Dim Sum Restaurant", "Dumpling Restaurant", "Sushi Restaurant", "Ramen Restaurant", "Vietnamese Restaurant", "Malay Restaurant", "Hotpot Restaurant", "Filipino Restaurant", "Taiwanese Restaurant", "Cantonese Restaurant"] 
    pattern = '|'.join(search_values)
    df1=df[df['Venue Category'].str.contains(pattern)]
    df1.reset_index(inplace=True)
    df1=df1[['Neighborhood','Latitude','Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']]
    return(df1)

In [14]:
asianrestaurants=Extract_only_Asian_Restaurant(restaurants)
asianrestaurants

,Neighborhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Eastchester,40.887556,-73.827806,Xing Lung Chinese Restaurant,40.888785,-73.831226,Chinese Restaurant
1,Pelham Parkway,40.857413,-73.854756,Mr. Q's Chinese Restaurant,40.855790,-73.855455,Chinese Restaurant
2,Pelham Parkway,40.857413,-73.854756,Kiraku Hibachi & Sushi,40.853356,-73.853629,Sushi Restaurant
3,Bedford Park,40.870185,-73.885512,Hung Hing Chinese Restaurant,40.871181,-73.886759,Chinese Restaurant
4,Bedford Park,40.870185,-73.885512,Choi Yuan - Chinese Restaurant,40.873078,-73.889086,Chinese Restaurant
5,Bedford Park,40.870185,-73.885512,New Xing Wang Chinese Restaurant,40.868875,-73.889355,Chinese Restaurant
6,Morris Heights,40.847898,-73.919672,Xing Sheng,40.846220,-73.918929,Chinese Restaurant
7,Fordham,40.860997,-73.896427,Panda Express,40.863001,-73.900894,Chinese Restaurant
8,Fordham,40.860997,-73.896427,Little China City,40.861811,-73.892046,Chinese Restaurant
9,Fordham,40.860997,-73.896427,China Wok II,40.860127,-73.893058,Chinese Restaurant


In [15]:
asianrestaurants.groupby('Neighborhood').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,1,1,1,1,1,1
Annadale,1,1,1,1,1,1
Arverne,1,1,1,1,1,1
Astoria,7,7,7,7,7,7
Astoria Heights,1,1,1,1,1,1
Auburndale,2,2,2,2,2,2
Bath Beach,7,7,7,7,7,7
Battery Park City,2,2,2,2,2,2
Bay Ridge,9,9,9,9,9,9


In [16]:
def one_hot_encoding(data_frame):
    asianrestaurants=data_frame
    one_hot = pd.get_dummies(asianrestaurants[['Venue Category']], prefix="", prefix_sep="")

    one_hot['Neighborhood'] = asianrestaurants['Neighborhood'] 

    fixed_columns = [one_hot.columns[-1]] + list(one_hot.columns[:-1])
    one_hot = one_hot[fixed_columns]
    print(one_hot.shape)
    return(one_hot)
one_hot=one_hot_encoding(asianrestaurants)
one_hot

(764, 18)


,Neighborhood,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Ramen Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vietnamese Restaurant
0,Eastchester,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Pelham Parkway,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Pelham Parkway,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Bedford Park,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bedford Park,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Bedford Park,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Morris Heights,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Fordham,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Fordham,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Fordham,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
def Group_by_Neighborhood(data_frame):
    one_hot=data_frame
    asianrestaurants_grouped = one_hot.groupby('Neighborhood').sum().reset_index()
    return(asianrestaurants_grouped)
asianrestaurants_grouped=Group_by_Neighborhood(data_frame=one_hot)
asianrestaurants_grouped

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Ramen Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vietnamese Restaurant
0,Allerton,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Annadale,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,Arverne,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Astoria,0,0,2,0,0,0,0,0,2,1,0,0,0,1,0,1,0
4,Astoria Heights,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Auburndale,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
6,Bath Beach,1,1,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0
7,Battery Park City,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,Bay Ridge,0,0,2,1,0,0,0,0,0,0,0,0,0,2,0,3,1
9,Bay Terrace,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [21]:
asianrestaurants_grouped[asianrestaurants_grouped['Chinese Restaurant']==asianrestaurants_grouped['Chinese Restaurant'].max()]

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Ramen Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vietnamese Restaurant
32,Chinatown,2,0,9,2,2,0,2,0,0,2,2,1,1,0,1,1,4


In [22]:
asianrestaurants_grouped[asianrestaurants_grouped['Sushi Restaurant']==asianrestaurants_grouped['Sushi Restaurant'].max()]

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Ramen Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vietnamese Restaurant
165,Turtle Bay,2,0,0,0,0,0,0,0,3,0,0,3,0,6,0,2,0


In [23]:
asianrestaurants_grouped[asianrestaurants_grouped['Japanese Restaurant']==asianrestaurants_grouped['Japanese Restaurant'].max()]

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Ramen Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vietnamese Restaurant
117,Murray Hill,1,0,2,0,0,0,0,0,5,21,0,1,1,2,0,1,1


In [24]:
asianrestaurants_grouped[asianrestaurants_grouped['Thai Restaurant']==asianrestaurants_grouped['Thai Restaurant'].max()]

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Ramen Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vietnamese Restaurant
58,Elmhurst,0,0,3,0,0,0,1,0,0,0,1,0,0,1,0,7,2


In [25]:
asianrestaurants_grouped[asianrestaurants_grouped['Korean Restaurant']==asianrestaurants_grouped['Korean Restaurant'].max()]

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Ramen Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vietnamese Restaurant
117,Murray Hill,1,0,2,0,0,0,0,0,5,21,0,1,1,2,0,1,1


In [30]:
one_hot.loc['total']=one_hot.sum()
one_hot

,Neighborhood,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Filipino Restaurant,Hotpot Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Korean Restaurant,Malay Restaurant,Ramen Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vietnamese Restaurant
0,Eastchester,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Pelham Parkway,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Pelham Parkway,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Bedford Park,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bedford Park,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Bedford Park,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Morris Heights,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Fordham,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Fordham,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Fordham,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
